In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data load and viewing

In [ ]:
sample_submission=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/sample_submission.csv")
train=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/train.csv")
test=pd.read_csv("/kaggle/input/tabular-playground-series-feb-2022/test.csv")

**Train data**

In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
print(f'Number of rows in train data: {train.shape[0]}')
print(f'Number of columns in train data: {train.shape[1]}')
print(f'Number of values in train data: {train.count().sum()}')
print(f'\033[91mNumber missing values in train data: {sum(train.isna().sum())}')
print(f'\033[91mNumber of duplicates in train data: {train.duplicated().sum()}')

In [ ]:
print(sorted(train.columns.unique()))
print(train['target'].unique())

**Test data**

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
print(f'Number of rows in test data: {test.shape[0]}')
print(f'Number of columns in test data: {test.shape[1]}')
print(f'Number of values in test data: {test.count().sum()}')
print(f'\033[91mNumber missing values in test data: {sum(test.isna().sum())}')
print(f'\033[91mNumber of duplicates in test data: {test.duplicated().sum()}')

In [ ]:
print(sorted(test.columns.unique()))

**Sample Submission**

In [ ]:
sample_submission.head()

# Data cleanup and sample weight

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

# Variable assignment and column drop

In [ ]:
trainX = train.drop(columns=['row_id', 'target'])
trainY = train[['target']]
testX = test.drop(columns=['row_id'])
testY = test[['row_id']]

# Model = RandomForestClassifier, ExtraTreesClassifier

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
trainY = encoder.fit_transform(trainY)

In [ ]:
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_predict

#check max_depth and other parameters
#modelRF = RandomForestClassifier(n_estimators=4000, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features='auto', 
#                               max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, n_jobs=-1, random_state=1000, verbose=4, max_samples=1)
modelET = ExtraTreesClassifier(n_estimators=500, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features='auto', 
                               max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=False, n_jobs=-1, random_state=1000, verbose=4, max_samples=1)

modelStack = StackingClassifier(estimators=[('ET1', modelET), ('ET2', modelET), ('ET3', modelET)], final_estimator=modelET)

modelStack.fit(trainX, trainY)

y_pred = modelStack.predict(testX)

# Submission

In [ ]:
y_pred = encoder.inverse_transform(y_pred)
submission = pd.DataFrame({'row_id': test.row_id, 'target': y_pred})
submission.to_csv('submission.csv', index=False)
submission.head()

# Visualization and accuracy

In [ ]:
from sklearn.metrics import accuracy_score

modelStack.score(trainX, trainY)

from sklearn.datasets import load_iris
iris = load_iris()

/# Model (can also use single decision tree)
/# from sklearn.ensemble import RandomForestClassifier
/# model = RandomForestClassifier(n_estimators=10)

/# Train
modelStack.fit(iris.data, iris.target)
/# Extract single tree
estimator = model.estimators_[5]

from sklearn.tree import export_graphviz
/# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = iris.feature_names,
                class_names = iris.target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

/# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

/# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')